In [1]:
import os
from configparser import ConfigParser

config = ConfigParser()
config.read('../conf.ini')
openai_token=config['secret']['openai_token']
os.environ['OPENAI_API_KEY'] = openai_token

# Chains
- 좀 더 복잡한 Application을 구동하기 위해서 사용
    - 구성 요소를 체인으로 함께 구성
    - Application을 디버스, 유지 관리가 훨씬 편해짐

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="Can you recommend 5 good names for {company} that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }))



1. Joyful Socks
2. Colorful Crew
3. Rainbow Hues
4. Bright Feet
5. Happy Toes


## How to

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0)
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(
    llm=chat, 
    prompt=PromptTemplate.from_template(prompt_template), 
    verbose=True # For Debugging
    )

llm_chain(inputs={"adjective": "corny"})
llm_chain.output_keys



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


['text']

In [7]:
# 모두 같은 명령어
llm_chain.run({"adjective": "corny"})
llm_chain.run("corny")

llm_chain("corny")
llm_chain({"adjective": "corny"})



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Tell me a corny joke

> Finished chain.


{'adjective': 'corny',
 'text': "Sure, here's a corny joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"}

### OpenAI Function

#### Structued Output

In [10]:
from pydantic import BaseModel, Field
from typing import Optional, Sequence

class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")
    
class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")

In [13]:
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage

# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for extracting information in structured formats."),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

chain = create_structured_output_chain(Person, llm, prompt, verbose=True)
chain.run("Sally is 13 and favorite food is Korean food")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Sally is 13 and favorite food is Korean food
Human: Tip: Make sure to answer in the correct format

> Finished chain.


Person(name='Sally', age=13, fav_food='Korean food')

In [14]:
chain = create_structured_output_chain(People, llm, prompt, verbose=True)
chain.run("Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally.
Human: Tip: Make sure to answer in the correct format

> Finished chain.


People(people=[Person(name='Sally', age=13, fav_food=''), Person(name='Joey', age=12, fav_food='spinach'), Person(name='Caroline', age=23, fav_food='')])

In [15]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "fav_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}
chain = create_structured_output_chain(json_schema, llm, prompt, verbose=True)
chain.run("Sally is 13")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input: Sally is 13
Human: Tip: Make sure to answer in the correct format

> Finished chain.


{'name': 'Sally', 'age': 13}

### Caching

In [8]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")
conversation.run("And the next 4?")

'The next four colors of a rainbow are green, blue, indigo, and violet.'

### Async API
- asyncio 라이브러리를 통한 체인의 비동기 지원

In [6]:
import asyncio
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def generate_serially():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="baseball equipment")
        print(resp)


async def async_generate(chain):
    resp = await chain.arun(product="baseball equipment")
    print(resp)


async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


s = time.perf_counter()
await generate_concurrently()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Async function executed in {elapsed:0.2f} seconds." + "\033[0m")

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")



Diamond Sports Gear.


Diamond Glove Sports.


Hit & Field Supplies.


Batty's Baseball Supplies


Batter's Edge.
Async function executed in 0.84 seconds.


Diamond Sports.


Diamond Sports Gear.


Batters and Blades.


Diamond Bat Company.


PowerPlay Baseball Gear.
Serial executed in 2.72 seconds.


## Foundational

### LLM